In [1]:
import tkinter as tk
from tkinter import ttk

# --- Enigma Components ---

class Rotor:
    """Represents a single rotor in the Enigma machine."""
    ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self, name: str, wiring: str, notch: str):
        """
        Initializes a rotor.

        Args:
            name: The name of the rotor (e.g., "I").
            wiring: A string representing the internal wiring.
            notch: A string of letters indicating the turnover points.
        """
        self.name = name
        self.wiring = [self.ALPHABET.find(char) for char in wiring]
        self.notch = set(notch)
        self.position = 0  # Current rotational position (0-25)

    def rotate(self):
        """Rotates the rotor one step."""
        self.position = (self.position + 1) % 26

    def at_notch(self) -> bool:
        """Checks if the current position is at a notch."""
        current_letter = self.ALPHABET[self.position]
        return current_letter in self.notch

    def forward(self, signal_in: int) -> int:
        """Passes a signal through the rotor (right to left)."""
        entry_point = (signal_in + self.position) % 26
        output_index = self.wiring[entry_point]
        return (output_index - self.position + 26) % 26

    def backward(self, signal_in: int) -> int:
        """Passes a signal through the rotor in reverse (left to right)."""
        entry_point = (signal_in + self.position) % 26
        input_index = self.wiring.index(entry_point)
        return (input_index - self.position + 26) % 26

class Reflector:
    """Represents the reflector in the Enigma machine."""
    ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self, name: str, mapping: str):
        """
        Initializes a reflector.

        Args:
            name: The name of the reflector (e.g., "B").
            mapping: A string representing the wiring (pairs of reflected letters).
        """
        self.name = name
        self.reflection_map = {}
        for i in range(0, 26, 2):
            a = mapping[i]
            b = mapping[i+1]
            self.reflection_map[a] = b
            self.reflection_map[b] = a

    def reflect(self, char: str) -> str:
        """Reflects a character."""
        return self.reflection_map[char]

class Plugboard:
    """Represents the plugboard for letter swaps."""
    ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self, pairs: str = ""):
        """
        Initializes the plugboard.

        Args:
            pairs: A string of letter pairs to swap (e.g., "AB CD").
        """
        self.mapping = {letter: letter for letter in self.ALPHABET}
        self.set_pairs(pairs)

    def set_pairs(self, pairs: str):
        """Sets the plugboard pairs."""
        for pair in pairs.upper().split():
            if len(pair) == 2 and pair[0] in self.ALPHABET and pair[1] in self.ALPHABET:
                a, b = pair
                self.mapping[a] = b
                self.mapping[b] = a

    def process(self, char: str) -> str:
        """Processes a character through the plugboard."""
        return self.mapping.get(char, char)

# --- Enigma Machine ---

class EnigmaMachine:
    """Simulates the Enigma machine."""
    ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self, rotors: list[Rotor], reflector: Reflector, plugboard: Plugboard):
        """
        Initializes the Enigma machine.

        Args:
            rotors: A list of three Rotor objects (right to left).
            reflector: A Reflector object.
            plugboard: A Plugboard object.
        """
        self.rotors = rotors
        self.reflector = reflector
        self.plugboard = plugboard

    def _advance_rotors(self):
        """Advances the rotors according to the Enigma stepping mechanism."""
        self.rotors[0].rotate()
        if self.rotors[0].at_notch():
            self.rotors[1].rotate()
            if self.rotors[1].at_notch():
                self.rotors[2].rotate()

    def encrypt_letter(self, letter: str) -> str:
        """Encrypts a single letter."""
        if letter not in self.ALPHABET:
            return letter

        self._advance_rotors()

        # Plugboard
        signal = self.plugboard.process(letter)
        signal_index = self.ALPHABET.find(signal)

        # Rotors (right to left)
        signal_index = self.rotors[2].forward(signal_index)
        signal_index = self.rotors[1].forward(signal_index)
        signal_index = self.rotors[0].forward(signal_index)

        # Reflector
        reflected_signal = self.reflector.reflect(self.ALPHABET[signal_index])
        reflected_index = self.ALPHABET.find(reflected_signal)

        # Rotors (left to right)
        reflected_index = self.rotors[0].backward(reflected_index)
        reflected_index = self.rotors[1].backward(reflected_index)
        reflected_index = self.rotors[2].backward(reflected_index)

        # Plugboard
        final_signal = self.plugboard.process(self.ALPHABET[reflected_index])
        return final_signal

    def encrypt_text(self, text: str) -> str:
        """Encrypts an entire text."""
        ciphertext = ""
        for char in text.upper():
            ciphertext += self.encrypt_letter(char)
        return ciphertext

# --- GUI ---

class EnigmaGUI:
    """Graphical user interface for the Enigma simulator."""

    ROTOR_CHOICES = ["I", "II", "III", "IV", "V"]
    REFLECTOR_CHOICES = ["B", "C"]
    ROTOR_DATA = {
        "I": ("EKMFLGDQVZNTOWYHXUSPAIBRCJ", "Q"),
        "II": ("AJDKSIRUXBLHWTMCQGZNPYFVOE", "E"),
        "III": ("BDFHJLCPRTXVZNYEIWGAKMUSQO", "V"),
        "IV": ("ESOVPZJAYQUIRHXLNFTGKDCMWB", "J"),
        "V": ("VZBRGITYUPSDNHLXAWMJQOFECK", "Z"),
    }
    REFLECTOR_DATA = {
        "B": "YRUHQSLDPXNGOKMIEBFZCWVJAT",
        "C": "FVPJIAOYEDRZXWGCTKUQSBNMHL",
    }

    def __init__(self, master: tk.Tk):
        """Initializes the Enigma GUI."""
        self.master = master
        master.title("Enigma Machine Simulator")

        # --- Rotor Configuration ---
        ttk.Label(master, text="Rotor 1:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.rotor1_combo = ttk.Combobox(master, values=self.ROTOR_CHOICES, state="readonly")
        self.rotor1_combo.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.rotor1_combo.set("III")
        self.rotor1_pos_label = ttk.Label(master, text="Position:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.rotor1_pos_entry = ttk.Entry(master, width=5)
        self.rotor1_pos_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        self.rotor1_pos_entry.insert(0, "A")

        ttk.Label(master, text="Rotor 2:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        self.rotor2_combo = ttk.Combobox(master, values=self.ROTOR_CHOICES, state="readonly")
        self.rotor2_combo.grid(row=0, column=3, padx=5, pady=5, sticky="ew")
        self.rotor2_combo.set("II")
        self.rotor2_pos_label = ttk.Label(master, text="Position:").grid(row=1, column=2, padx=5, pady=5, sticky="w")
        self.rotor2_pos_entry = ttk.Entry(master, width=5)
        self.rotor2_pos_entry.grid(row=1, column=3, padx=5, pady=5, sticky="ew")
        self.rotor2_pos_entry.insert(0, "A")

        ttk.Label(master, text="Rotor 3:").grid(row=0, column=4, padx=5, pady=5, sticky="w")
        self.rotor3_combo = ttk.Combobox(master, values=self.ROTOR_CHOICES, state="readonly")
        self.rotor3_combo.grid(row=0, column=5, padx=5, pady=5, sticky="ew")
        self.rotor3_combo.set("I")
        self.rotor3_pos_label = ttk.Label(master, text="Position:").grid(row=1, column=4, padx=5, pady=5, sticky="w")
        self.rotor3_pos_entry = ttk.Entry(master, width=5)
        self.rotor3_pos_entry.grid(row=1, column=5, padx=5, pady=5, sticky="ew")
        self.rotor3_pos_entry.insert(0, "A")

        # --- Reflector Configuration ---
        ttk.Label(master, text="Reflector:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.reflector_combo = ttk.Combobox(master, values=self.REFLECTOR_CHOICES, state="readonly")
        self.reflector_combo.grid(row=2, column=1, columnspan=5, padx=5, pady=5, sticky="ew")
        self.reflector_combo.set("B")

        # --- Plugboard Configuration ---
        ttk.Label(master, text="Plugboard (e.g., AB CD EF):").grid(row=3, column=0, columnspan=6, padx=5, pady=5, sticky="w")
        self.plugboard_entry = ttk.Entry(master, width=50)
        self.plugboard_entry.grid(row=4, column=0, columnspan=6, padx=5, pady=5, sticky="ew")

        # --- Text Input/Output ---
        ttk.Label(master, text="Plaintext:").grid(row=5, column=0, columnspan=6, padx=5, pady=5, sticky="w")
        self.input_text = tk.Text(master, height=5, width=50)
        self.input_text.grid(row=6, column=0, columnspan=6, padx=5, pady=5, sticky="nsew")

        ttk.Label(master, text="Ciphertext:").grid(row=7, column=0, columnspan=6, padx=5, pady=5, sticky="w")
        self.output_text = tk.Text(master, height=5, width=50, state=tk.DISABLED)
        self.output_text.grid(row=8, column=0, columnspan=6, padx=5, pady=5, sticky="nsew")

        # --- Encrypt Button ---
        encrypt_button = ttk.Button(master, text="Encrypt/Decrypt", command=self.process_text)
        encrypt_button.grid(row=9, column=0, columnspan=6, pady=10)

        # --- Configure Grid Layout ---
        for i in range(6):
            master.grid_columnconfigure(i, weight=1)
        master.grid_rowconfigure(6, weight=1)
        master.grid_rowconfigure(8, weight=1)

    def _create_enigma_machine(self) -> EnigmaMachine:
        """Creates an EnigmaMachine instance based on the GUI settings."""
        # Get selected rotor types and initial positions
        rotor1_name = self.rotor1_combo.get()
        rotor2_name = self.rotor2_combo.get()
        rotor3_name = self.rotor3_combo.get()

        try:
            rotor1_pos = self.rotor1_pos_entry.get().upper()[0]
            rotor2_pos = self.rotor2_pos_entry.get().upper()[0]
            rotor3_pos = self.rotor3_pos_entry.get().upper()[0]
            rotor1_start = Rotor.ALPHABET.find(rotor1_pos)
            rotor2_start = Rotor.ALPHABET.find(rotor2_pos)
            rotor3_start = Rotor.ALPHABET.find(rotor3_pos)
        except IndexError:
            rotor1_start = 0
            rotor2_start = 0
            rotor3_start = 0

        # Create Rotor objects
        wiring1, notch1 = self.ROTOR_DATA[rotor1_name]
        rotor1 = Rotor(rotor1_name, wiring1, notch1)
        rotor1.position = rotor1_start

        wiring2, notch2 = self.ROTOR_DATA[rotor2_name]
        rotor2 = Rotor(rotor2_name, wiring2, notch2)
        rotor2.position = rotor2_start

        wiring3, notch3 = self.ROTOR_DATA[rotor3_name]
        rotor3 = Rotor(rotor3_name, wiring3, notch3)
        rotor3.position = rotor3_start

        rotors = [rotor3, rotor2, rotor1]  # Order is right to left in processing

        # Create Reflector object
        reflector_name = self.reflector_combo.get()
        reflector_mapping = self.REFLECTOR_DATA[reflector_name]
        reflector = Reflector(reflector_name, reflector_mapping)

        # Create Plugboard object
        plugboard_settings = self.plugboard_entry.get()
        plugboard = Plugboard(plugboard_settings)

        return EnigmaMachine(rotors, reflector, plugboard)

    def process_text(self):
        """Encrypts or decrypts the text in the input area."""
        enigma = self._create_enigma_machine()
        plaintext = self.input_text.get("1.0", tk.END).strip()
        ciphertext = enigma.encrypt_text(plaintext)

        self.output_text.config(state=tk.NORMAL)
        self.output_text.delete("1.0", tk.END)
        self.output_text.insert("1.0", ciphertext)
        self.output_text.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    gui = EnigmaGUI(root)
    root.mainloop()